<a href="https://colab.research.google.com/github/hiris25/Tierpsy-Tracking-Analysis/blob/main/Timeseries_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time series analysis
This script aligns time series to a stimulus/event using a munually produced list of trajectory IDs and event time stamps.

In [ ]:
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns

### Import files

In [ ]:
#import times
times = pd.read_csv('your events.csv')

#Import all HDF5 files from onestrain (folder) into one variableand extract tables from them one by one. Be careful to be in the filepath, add r'C: to file paths if run in Windows
datafolder = Path('folder') 
# this structure allows you to not have to change the rest of the paths in the code, it adapts them to Windows and Mac

Strain1 = sorted(list(datafolder.glob('*_featuresN.hdf5')))
#import all HDF5 FeaturesN files for interesting strain sorted in alphabetical order

In [ ]:
#Select timeseries data from files, adding all files from different videos for the same strain onto the same dataframe. To speed this part and avoid having to repeat the code I could create a def function and use map to apply it over the two Strain folders

Strain1_feat = pd.DataFrame() #dataframe to store information 
temporary1 = []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

for iteration, i in enumerate(Strain1): 
    data = pd.read_hdf(i, 'timeseries_data') #select all timeseries_data tables from the hdf5 documents in the folder
    data = data.assign(Video = iteration)
    data = data.assign(File = i)
    temporary1.append(data) #append all data
Strain1_feat = pd.concat(temporary1, ignore_index = True) #concatenate next to each other all the tables for all the files in the folder

In [ ]:
#keep only speed (or any other feature(s) you want)

Strain1_feat_speed = pd.DataFrame()

Strain1_feat_speed['speed'] = Strain1_feat['speed']

#keep optional extra features
Strain1_feat_speed['speed_head_tip'] = Strain1_feat['speed_head_tip']
Strain1_feat_speed['relative_to_neck_radial_velocity_head_tip'] = Strain1_feat['relative_to_neck_radial_velocity_head_tip']
Strain1_feat_speed['relative_to_head_base_radial_velocity_head_tip'] = Strain1_feat['relative_to_head_base_radial_velocity_head_tip']
Strain1_feat_speed['angular_velocity_head_tip'] = Strain1_feat['angular_velocity_head_tip']
Strain1_feat_speed['relative_to_neck_angular_velocity_head_tip'] = Strain1_feat['relative_to_neck_angular_velocity_head_tip']

#keep important features
Strain1_feat_speed['Video'] = Strain1_feat['Video']
Strain1_feat_speed['worm_index'] = Strain1_feat['worm_index']
Strain1_feat_speed['timestamp'] = Strain1_feat['timestamp']
Strain1_feat_speed['File'] = Strain1_feat['File']

Strain1_feat_speed = Strain1_feat_speed.astype('int64', errors='ignore')

videos are given ID's starting at 0, label the video number with the correct strain, videos should be in the alphabetical order that they appeared in the folder, DOUBLE CHECK THIS IS CORRECT before continuing

In [ ]:
#create strain dictionary 
genotypes = {0:'N2', 1:'N2', 2:'strain1', 3:'strain2', 4:'strain2', 5:'strain2', 6:'strain2', 7:'strain2', 8:'strain2', 9:'strain2', 10:'strain2'}
#map dictionary onto df
Strain1_feat_speed['strain'] = Strain1_feat_speed['Video'].map(genotypes)

In [ ]:
times = times.drop(columns=['strain'])

### Merge encounters

In [ ]:
encounters = pd.merge(Strain1_feat_speed, times, how="left", on=["Video", "timestamp", "worm_index"])

#check that encounter frames have been found
encounters['encounter'].value_counts()

In [ ]:
# select how long (frames) before and after you want (10fps)
t = 90

#get only row index with encounter = 1
index = encounters.index
condition = encounters["encounter"] == 1
excounter_indx = index[condition]
excounter_indx_list = excounter_indx.tolist()

#new df for before and after encounter
encounters_before_after = pd.DataFrame()

for i in excounter_indx_list:
  a = encounters.iloc[i-t:i+t]
  a.reset_index(inplace=True)
  encounters_before_after = encounters_before_after.append(a)

encounters_before_after = encounters_before_after.drop(columns = 'index')
encounters_before_after.reset_index(inplace=True)
encounters_before_after = encounters_before_after.rename(columns = {'index':'time_norm'})
encounters_before_after['time_norm'] = encounters_before_after['time_norm'] - t

encounters_before_after['speed_abs'] = encounters_before_after['speed'].abs()

### Plotting

In [ ]:
sns.lineplot(data=encounters_before_after, x='time_norm', y='speed', hue="worm_index")

In [ ]:
sns.lineplot(data=encounters_before_after, x='time_norm', y='speed', hue='strain')